Cross Validation of SARIMAX Forecasting Nuitees dans l'hôtellerie Provence-Alpes-Côte d'Azur PAC 
Set "stepwise = true" in auto-arima.
Include 3 exogenous variables (meteo,GDP,holidays) in training & test datasets
Predict GDP with auto-arima for 2016 & 2017 and include as exogenous in training&test datasets
Datasets https://github.com/jinnyto/touristcast/tree/datasets/datasets

In [12]:
import pandas as pd
import numpy as np
import datetime 
import matplotlib.pyplot as plt
import pyramid
%matplotlib inline
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt
def seasonal_MASE(truth, forecast, seasonality=1):    
    period = truth.shape[0] # T
    # print(period)
    forecast_errors = np.abs(truth - forecast)
    # print(forecast_errors)
    mean_absolute_forecast_error = np.sum(forecast_errors) / period
    # print(mean_absolute_forecast_error)

    naive_period = truth.shape[0] - seasonality # T - m
    # print(naive_period)
    # print(truth[seasonality:])
    # print(truth[:period - seasonality])
    naive_errors = np.abs(truth[seasonality:] - truth[:period - seasonality])
    mean_absolute_naive_error = np.sum(naive_errors) / naive_period
    
    return mean_absolute_forecast_error / mean_absolute_naive_error

In [13]:
dn = pd.read_csv('./datasets/nights_2010-2017.csv',parse_dates=['date'], index_col='date',usecols=[*range(1, 15)])
dm = pd.read_csv('./datasets/avgtemp_2010-2017.csv',parse_dates=['date'],index_col='date',usecols=[*range(1, 15)])
dh = pd.read_csv('./datasets/daysoff_2010-2017.csv',parse_dates=['date'],index_col='date',usecols=['date','daysoff'])
dg = pd.read_csv('./datasets/regionalGDP_2010-2015.csv',parse_dates=['date'],index_col='date',usecols=[*range(1, 15)])
dr = pd.concat([dn, dm,dh,dg], axis=1)
df = dr.dropna()
df = df[['nights_PAC','avgtemp_PAC','gdp_PAC','daysoff']]
exogenous= np.array(df[['avgtemp_PAC','gdp_PAC','daysoff']])
y = np.array(df.nights_PAC)

In [14]:
from pyramid.arima import auto_arima
tc_fit = auto_arima(y=y,exogenous=exogenous,start_p=1, start_q=1, max_p=3, max_q=3, m=12,
                    start_P=1, seasonal=True, d=1, D=1, trace=True,
                    error_action='ignore',  # don't want to know if an order does not work
                    suppress_warnings=True,  # don't want convergence warnings
                    stepwise=True)
tc_fit.summary()

Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=687.799, BIC=706.497, Fit time=2.184 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=709.629, BIC=720.017, Fit time=0.157 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=699.327, BIC=713.870, Fit time=1.448 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=687.750, BIC=702.293, Fit time=1.634 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=689.214, BIC=705.834, Fit time=1.952 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=698.393, BIC=710.858, Fit time=0.670 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 2, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=686.360, BIC=702.981, Fit time=1.958 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(0, 1, 1, 12); AIC=694.7

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                 
==========================================================================================
Dep. Variable:                                  y   No. Observations:                   72
Model:             SARIMAX(1, 1, 1)x(0, 1, 1, 12)   Log Likelihood                -335.180
Date:                            Wed, 29 Aug 2018   AIC                            686.360
Time:                                    13:34:53   BIC                            702.981
Sample:                                         0   HQIC                           692.848
                                             - 72                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.3731      0.779     -0.479      0.632      -1.900       1.154
x1            -0.2321      6.276     -0.037      0.971     -12.533      12.069
x2             0.0217      0.013      1.650      0.099      -0.004       0.047
x3            25.4305      8.493      2.994      0.003       8.784      42.077
ar.L1          0.3032      0.174      1.744      0.081      -0.038       0.644
ma.L1         -0.9995     15.939     -0.063      0.950     -32.239      30.240
ma.S.L12      -0.6669      0.296     -2.252      0.024      -1.247      -0.087
sigma2      4062.7568   6.49e+04      0.063      0.950   -1.23e+05    1.31e+05
===================================================================================
Ljung-Box (Q):                       30.70   Jarque-Bera (JB):                 7.14
Prob(Q):                              0.85   Prob(JB):                         0.03
Heteroskedasticity (H):               1.37   Skew:                            -0.41
Prob(H) (two-sided):                  0.49   Kurtosis:                         4.49
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [20]:
mc_fit = auto_arima(y=np.array(df.gdp_PAC),start_p=0, start_q=0, max_p=3, max_q=3, m=12,
                    start_P=1, seasonal=False, d=1, D=1, trace=True,
                    error_action='ignore',  # don't want to know if an order does not work
                    suppress_warnings=True,  # don't want convergence warnings
                    stepwise=True)
mc_fit.summary()

Fit ARIMA: order=(0, 1, 0); AIC=1139.416, BIC=1143.942, Fit time=0.023 seconds
Fit ARIMA: order=(1, 1, 0); AIC=1141.212, BIC=1148.000, Fit time=0.162 seconds
Fit ARIMA: order=(0, 1, 1); AIC=1141.184, BIC=1147.972, Fit time=0.036 seconds
Fit ARIMA: order=(1, 1, 1); AIC=nan, BIC=nan, Fit time=nan seconds
Total fit time: 0.249 seconds


<class 'statsmodels.iolib.summary.Summary'>
"""
                             ARIMA Model Results                              
==============================================================================
Dep. Variable:                    D.y   No. Observations:                   71
Model:                 ARIMA(0, 1, 0)   Log Likelihood                -567.708
Method:                           css   S.D. of innovations            718.346
Date:                Wed, 29 Aug 2018   AIC                           1139.416
Time:                        13:39:52   BIC                           1143.942
Sample:                             1   HQIC                          1141.216
                                                                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        165.7243     85.252      1.944      0.056      -1.367     332.815
==============================================================================
"""

In [21]:
mc_fit.fit(df.gdp_PAC)

ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(0, 1, 0),
   out_of_sample_size=0, scoring='mse', scoring_args={},
   seasonal_order=None, solver='lbfgs', start_params=None,
   suppress_warnings=True, transparams=True, trend='c')

In [30]:
next_24 = mc_fit.predict(n_periods=24)
dr.loc['2016-01-01':'2017-12-01','gdp_PAC'] = next_24

In [36]:
smase_fold= []
startyear = '2010-01-01'
for index in range(2011,2018):
    nextyear = str(index)+'-01-01'
    predictyear = str(index+1)+'-01-01'
    dftrain = dr[(dr.index>=startyear)&(dr.index<nextyear)]
    train_nights  = np.array(dftrain.nights_PAC)
    train_exogenous =  np.array(dftrain[['avgtemp_PAC','gdp_PAC','daysoff']])
    dftest = dr[(dr.index>=nextyear)&(dr.index<predictyear)]
    test_nights = np.array(dftest.nights_PAC)
    test_exogenous =  np.array(dftest[['avgtemp_PAC','gdp_PAC','daysoff']])
    tc_future_forecast = tc_fit.predict(n_periods=12,exogenous = test_exogenous)
    smase_fold.append(seasonal_MASE(test_nights,tc_future_forecast))

print(smase_fold)
np.sum(smase_fold) / len(smase_fold)

[0.5930528923169276, 0.44105396972192534, 0.3757730158741758, 0.21937170795945204, 0.16015441233073952, 0.2103644129115874, 0.14933392615478977]


0.3070149053242282